<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 12</h1>

This analysis draws extensively from [Edwards and Roff (2010)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0012157) and was featured on the Fall 2022 midterm exam in ECON 140R at the University of California, Berkeley.

Consider the dataset `cpp_sample.dta` drawn from the multi-site U.S. Collaborative Perinatal Project (CPP), a panel survey spanning 7 years of children’s lives. The study began with pregnant women who were recruited at university hospitals between 1959 and 1965. They and their children were followed for 7 years of the children's lives, with several waves of questions and neurocognitive tests administered along the way.

There is potentially a lot more to be said here, about how the panel was reinterviewed, how much attrition there was, and so on. Here, we are going to take a very simple approach. We will examine data from each wave separately, ignoring the panel structure but in a way that avoids big econometric problems.

How can we draw inferences from observational data about effects of "treatments" on outcomes we care about? Multivariate regression methods &mdash; using `lm()` in R, for example &mdash; can help. But a critical issue is whether we can control sufficiently for <b>omitted variable bias (OVB)</b>. In this dataset, which is publicly available, we can walk through several examples of omitted variables that biased published results.

<h2>Learning objectives</h2>

1. Examine a real-world dataset and published result about the effect of paternal age ($x$) on kids' test scores ($y$)
2. Explore the role of omitted family variables
3. Use the omitted variable bias (OVB) formula

Let's start by looking at the dataset, running a regression, and talking and writing about what we see.

<h2>Variables</h2>

For <b>outcome variables</b> $y$, we can look at the same six CPP measures of children’s <i>neurocognitive development</i> that were examined by [Saha et al. (2009)](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1000040), two measures at each follow-up age: 8 mo, 4 y, and 7 y. In order, these include 
* the Bayley Mental Scale `bayleymental` and Bayley Motor Scale `bayleymotor` for Infant Development
* the Stanford Binet Intelligence Scale Form L-M `stanfordbinet` and the Graham-Ernhart Block Sort Test `grahamernhart` 
* the Wechsler Intelligence Scale for Children (WISC) Full Scale IQ `wiscfulliq` and the Wide Range Achievement Test (WRAT) of Reading `wratreading`

The CPP data, drawn from the enhanced electronic datasets distributed by the Johns Hopkins School of Public Health, include WRAT scores that are raw rather than normed, with a mean around 35 rather than 100. [Edwards and Roff (2010)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0012157) found no qualitative differences between results using these raw scores and the normed scores used by [Saha et al. (2009)](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1000040). (In theory there should be no qualitative differences.)

The <b>treatment variable</b> of interest is <b>paternal age</b>, labeled `fathage` in the dataset. 

[Saha et al. (2009)](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1000040) are interested in whether there are developmental risks to children of older fathers. As they discuss, previous research suggests that advanced paternal age, which is connected to a higher probability of copy error mutations in sperm, appears to be associated with fetal death, rare congenital conditions, neurological and neuropsychiatric conditions, schizophrenia, and autism spectrum disorder. Advanced <i>maternal age</i> carries known risk factors, and this and other studies aimed to explore what advanced paternal age may bring with it, during a period when childbearing is generally being postponed to later in life, especially among the college educated.

The regression equation takes this general form:

$$
y_i = \alpha + \beta \cdot fathage_i + \sum \gamma^j \cdot z^j_i + \epsilon_i
$$

where $y_i$ is a neurocognitive test score at a particular are; $\beta$ is the treatment effect of paternal age `fathage` on $y_i$; and each $z^j_i$ is a "control variable" or background characteristic, with an effect $\gamma^j$. 

Many other variables measure things that are likely to affect children's neurocognitive scores, of course. [Saha et al. (2009)](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1000040) considered the following set:

* `mothage` 
    * Mother's age at birth
* `fathage` 
    * Father's age at birth
* `childage8mo` `childage4yr` `childage7yr`
    * Child's age at reinterview wave
* `gestationwks`    
    * Child's gestation in weeks
* `male`
    * 0/1 Child is male
* `mothblack`
    * 0/1 Mother is Black (or African American)
* `mothasian`
    * 0/1 Mother is Asian American
* `mothpuert` 
    * 0/1 Mother is Puerto Rican
* `mothother` 
    * 0/1 Mother is of other race, not White non-Hispanic
* `mothsingle` 
    * 0/1 Mother's marital status: single
* `mothcommonl` 
    * 0/1 mother's marital status: common-law married 
* `mothwidowed` 
    * 0/1 mother's marital status: widowed 
* `mothdivorce`   
    * 0/1 mother's marital status: divorced 
* `mothseparat`  
    * 0/1 mother's marital status: separated 
* `socioeconindex`
    * 0/1 family's [Duncan socioeconomic index](https://usa.ipums.org/usa-action/variables/SEI#description_section) 
* `mothmentill`    
    * 0/1 mother's mental health history (mentally ill) 
* `fathmentill`  
    * 0/1 father's mental health history (mentally ill) 

Wow, what a list! But ... is it enough? <i>Are there other critical variables that are still omitted?</i>

<b>SPOILER ALERT:</b>  <i>Yes. There are still critical variables that are still omitted.</i>

<hr>


Let's load up <b>haven</b> and <b>tidyverse</b>

In [ ]:
library(haven)
library(tidyverse)

In [ ]:
options(scipen=999)

And let's load in the dataset:

In [ ]:
cpp_sample <- read_dta("data/cpp_sample.dta")

In [ ]:
head(cpp_sample)

[Saha et al. (2009)](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1000040) examined six different neurocognitive test scores. Let's choose one to focus on. Let's choose the Wechsler Intelligence Scale for Children (WISC), measured at age 7. It's useful to look at the summary of the variable, which reveals the units and average value.

In [ ]:
summary(cpp_sample$wiscfulliq)

<h2>Walkthrough 1: The kitchen sink(s)</h2>

Let's start the way a research paper might proceed. We will run a "short regression" that matches the published work we are examining, and then we will run two "long regressions" that includes <i>several</i> additional variables that we think matter. In other words, the long regressions will include the kitchen sink, or two kitchen sinks.

Following what [Saha et al. (2009)](https://journals.plos.org/plosmedicine/article?id=10.1371/journal.pmed.1000040) did, let's start with a "short regression" that includes just a few basic characteristics: father's age, mother's age, child's age at test, gestational age (weeks spent in utero), child's gender, and mother's race/ethnicity.

In [ ]:
wisc_reg1 <- lm(wiscfulliq ~ fathage + 
                mothage + 
                childage7yr + gestationwks + male +
                mothblack + mothasian + mothpuert + mothother, 
                data = cpp_sample)
summary(wisc_reg1)

<font color="blue">Write about what you see here. Describe and discuss the "treatment effect" of `fathage`. What about the effect associated with `mothage`? How do these two coefficients compare? Are they statistically significant? What are their signs? Are they large relative to the average level of `wiscfulliq`? If both increase, roughly what happens to the kids' test score (which is the outcome variable)?</font>

<font color="blue">
Describe the nature of this study. How are "treatment groups" &mdash; children with older dads &mdash; and "control groups" &mdash; children with younger dads &mdash; assigned? Randomly? In some other way? If the assignment is not random, why do you think children might have older dads?
</font>

<h3>Kitchen sink #1</h3>

Now let's run a second regression, a longer one, with more control variables on the right-hand side. Like Saha et al. did next, let's also control for mother's marital status, the family socioeconomic index (a single-valued function of income, education, and occupation &mdash; these are commonly seen in sociology), and 0/1 indicators of mother's and father's having any history of mental illness.

In [ ]:
wisc_reg2 <- lm(wiscfulliq ~ fathage + 
                mothage + 
                childage7yr + gestationwks + male +
                mothblack + mothasian + mothpuert + mothother + 
                mothsingle + mothcommonl + mothwidowed + mothdivorce + mothseparat +
                socioeconindex + mothmentill + fathmentill, 
                data = cpp_sample)
summary(wisc_reg2)

<font color="blue">Write about what you see here. Describe and discuss the "treatment effect" of `fathage`. What about the effect associated with `mothage`? How do these two coefficients compare? Are they statistically significant? What are their signs? Are they large relative to the average level of `wiscfulliq`?</font>

<h3>Kitchen sink #2</h3>

Let's now run a third regression, with an even bigger kitchen sink. Here, we now add the child's birth weight, the mother's years of education, the father's years of education, the child's total number of siblings, and the child's number of younger siblings.

In [ ]:
wisc_reg3 <- lm(wiscfulliq ~ fathage + mothage + 
                childage7yr + gestationwks + male +
                mothblack + mothasian + mothpuert + mothother + 
                mothsingle + mothcommonl + mothwidowed + mothdivorce + mothseparat +
                socioeconindex + mothmentill + fathmentill +
                birthwt + 
                mothyrseduc + 
                fathyrseduc + 
                totsibs + youngersibs
                , data = cpp_sample)
summary(wisc_reg3)

<font color="blue">Write about what you see here. Describe and discuss the "treatment effect" of `fathage`. What about the effect associated with `mothage`? How do these two coefficients compare? Are they statistically significant? What are their signs? Are they large relative to the average level of `wiscfulliq`?</font>

<hr>

<h2>Walkthrough 2: The Fall 2022 Midterm</h2>
    
For the Fall 2022 midterm, I ran a bunch of regressions to identify the one omitted variable that seemed to tell most of the OVB story. When you have one omitted variable, you can run a single auxiliary regression and get a clean statement using the OVB formula.

Here is the short regression, the same as `wisc_reg1` above:

In [ ]:
wisc_reg_short <- lm(wiscfulliq ~ fathage + 
                     mothage + 
                     childage7yr + gestationwks + male +
                     mothblack + mothasian + mothpuert + mothother,
                     data = cpp_sample)
summary(wisc_reg_short)

Now here is a more parsimonious long regression, where we have only added `fathyrseduc` and nothing further. Witness the damage that it alone can do:

In [ ]:
wisc_reg_long <- lm(wiscfulliq ~ fathage + 
                    mothage + 
                    fathyrseduc + 
                    childage7yr + gestationwks + male +
                    mothblack + mothasian + mothpuert + mothother, 
                    data = cpp_sample)
summary(wisc_reg_long)

And here is the auxiliary regression, of the omitted variable from the long regression, `fathyrseduc`, on the treatment variable of interest and all the other variables from the long regression:

In [ ]:
wisc_reg_aux <- lm(fathyrseduc ~ fathage + 
                   mothage + 
                   childage7yr + gestationwks + male +
                   mothblack + mothasian + mothpuert + mothother, 
                   data = cpp_sample)
summary(wisc_reg_aux)

In symbolic math, here is what we have done, using $\lambda$ as the coefficient on the omitted variable in the long regression. (Sometimes we use $\gamma$.)

$$
\begin{aligned}
wisc_i & = \alpha^s + \beta^s \ fathage_i & + B^s X_i + \epsilon^s_i
\\
wisc_i & = \alpha^l + \beta^l \ fathage_i + \lambda \ fathyrseduc_i & + B^l X_i + \epsilon^l_i
\\
fathyrseduc_i & = \pi_0 + \pi_1 \ fathage_i   & + B^a X_i + \epsilon^a_i
\end{aligned}
$$

And the OVB formula tells us that 

$$
\beta^s - \beta^l = \pi_1 \times \lambda 
$$

In [ ]:
lambda <- wisc_reg_long$coefficients["fathyrseduc"]
pi_1 <- wisc_reg_aux$coefficients["fathage"]

cat("Lambda =", lambda, "\n")
cat("Pi_1 = ", pi_1,"\n")

In [ ]:
cat("Beta_short - Beta_long =", wisc_reg_short$coefficients["fathage"] - wisc_reg_long$coefficients["fathage"], "\n")

cat("OVB formula =", pi_1 * lambda, "\n")

A small difference emerges here, likely due to floating point rounding. On the exam, anything around $OVB = -0.16$ or so received full credit. It's not hard to eyeball what $OVB$ is here, because $\pi_1 \approx -0.1$.

<h3>Summary</h3>

Paternal age, `fathage`, is correlated with many things in the CPP data. One of them that really stands out is paternal education, `fathyrseduc`, and in the auxiliary regression, we see that $\pi_1 \approx -0.1$. Each additional year of paternal age was associated with a reduction in paternal education of about a tenth of a year.

Why? In a cross section especially in the earlier time period of the CPP, older dads are likely to be the ones <i>still</i> having children. Dad's education and socioeconomic status would likely reduce family size, possibly by raising desired spending and education per child, which is sometimes called "child quality."

Further, we also see that paternal education, `fathyrseduc`, exerts a positive influence on kids' test scores. In the long regression, $\lambda \approx 1.6$, meaning a little over an additional point and a half on the test for each additional year of paternal education. 

Why? Paternal and maternal education could proxy for time investments in the child, or they might proxy for biological inheritance of "smarts" or something. Family economists usually think it's the former, but it could be nature or nurture. Either way, we see a positive relationship there.

<h3>Omitted variable or bad control?</h3>

It does not seem plausible at all that sperm mutations cause lower test scores for kids through the channel of paternal education, which is likely determined far in advance of deleterious mutations in sperm. If that view is correct, then if paternal education is not controlled for, it is an omitted variable that is negatively correlated with paternal age and positively correlated with kids' test scores. That negative bias creates a coefficient on paternal age in the short regression, $\beta^s$, which is too negative. 

We cannot reject that $\beta^l = 0$, which is likely a better estimate of the "true" causal effect of paternal age and sperm mutation on children's test scores.  

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>